In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("cc-est2023-alldata-12.csv")

In [4]:
counties = ["Broward County","Duval County","Hillsborough County","Miami-Dade County"]

In [5]:
#filter to just 2023
df_2023 = df[df.YEAR == 5]

In [6]:
df_2023.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
76,50,12,1,Florida,Alachua County,5,0,285994,137792,148202,...,15866,16984,1547,1501,400,403,329,399,99,85
77,50,12,1,Florida,Alachua County,5,1,13197,6756,6441,...,759,758,165,137,44,44,35,35,15,5
78,50,12,1,Florida,Alachua County,5,2,14234,7212,7022,...,805,829,132,135,21,30,26,37,9,9
79,50,12,1,Florida,Alachua County,5,3,14512,7367,7145,...,815,831,136,146,23,29,45,48,8,4
80,50,12,1,Florida,Alachua County,5,4,25938,12181,13757,...,1607,1890,153,173,22,40,39,58,10,10


In [7]:
df_2023_populations = df_2023[(df_2023.AGEGRP == 0) & (df_2023.CTYNAME.isin(counties))]

In [8]:
df_2023_populations 

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
551,50,12,11,Florida,Broward County,5,0,1962531,963118,999413,...,293924,304885,23800,25126,4362,3926,3467,3916,1152,1116
1406,50,12,31,Florida,Duval County,5,0,1030822,499527,531295,...,55828,54919,8939,9119,1773,1810,1641,1601,429,411
2641,50,12,57,Florida,Hillsborough County,5,0,1535564,756860,778704,...,212951,210526,21165,21446,4917,4241,2633,2715,796,891
4066,50,12,86,Florida,Miami-Dade County,5,0,2686867,1320212,1366655,...,866573,913042,38082,37964,5728,5211,3709,3775,902,1052


In [9]:
#subset data to just the 4 counties we're interested in for the voting populations (20+)
df_2023_voting_populations = df_2023[(df_2023.AGEGRP.isin(range(5,19))) & (df_2023.CTYNAME.isin(counties))]

In [38]:
df_2023_voting_populations.to_csv("long_census.csv", index=False)

In [15]:
def weighted_average(nums, weights):
    return sum(x * y for x, y in zip(nums, weights)) / sum(weights)

In [19]:
Broward_age = weighted_average(df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Broward County"].AGEGRP, df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Broward County"].TOT_POP)
Duval_age = weighted_average(df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Duval County"].AGEGRP, df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Duval County"].TOT_POP)
Miami_age = weighted_average(df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Miami-Dade County"].AGEGRP, df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Miami-Dade County"].TOT_POP)
Hillsborough_age = weighted_average(df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Hillsborough County"].AGEGRP, df_2023_voting_populations[df_2023_voting_populations["CTYNAME"]=="Hillsborough County"].TOT_POP)

In [21]:
#group population totals by county
df_2023_voting_populations_grouped = df_2023_voting_populations.groupby('CTYNAME').sum()

In [23]:
#copy in the weighed avg calculations
df_2023_voting_populations_grouped['AGEGRP'] = [Broward_age,Duval_age,Hillsborough_age,Miami_age]

In [26]:
#translate the weighed avg to the index in reference file "CC-EST2023-ALLDATA.pdf"
df_2023_voting_populations_grouped['AGEGRP'] = ['45-49 years','45-49 years','45-49 years','45-49 years']

In [32]:

df_2023_voting_populations_grouped=df_2023_voting_populations_grouped.reset_index()

In [33]:
#create csv
df_2023_voting_populations_grouped.to_csv("census_data.csv", index=False)